In [1]:
import numpy, time
from typing import Dict, List, Tuple

In [6]:
def generate_row_compensation_space(previous_row: List,
                                    admissible_range: Tuple[int, int],
                                    max_vector_size: int,
                                    row_position: int,
                                    recursive_step: int = 0,
                                    row_list: List = None) -> List[int]:
    """
    A generator which builds are possible vectors of a certain size (max_vector_size) that are increasing, where its
    entries are taken from a range (admissible_range, representing a discrete interval [a,b]) and component-wise are 
    bigger than a given vector (previous_row).

    Args:
        previous_row: A list of integers, representing the vector whose entries are the lower-bounds of the generated vectors.
        admissible_range: A tuple of two integers, representing the interval from which the candidate values will be taken.
        max_vector_size: An integer, representing the size of the vector to be generated.
        recursive_step: An integer, representing the number of recursive calls. It is used as the stop criterion.
        row_list: A list of integers, representing the generated vector.

    Returns:
        A list of integers, representing the vector with size max_vector_size and which entries are in increasing order.
    """
    assert max_vector_size <= len(previous_row), "Number of entries of the generated vector is greater than the size of the bound vector. Expected to be less than or equal to its size."
    
    min_value = admissible_range[0]
    max_value = admissible_range[1]
    
    if row_list is None:
        row_list = []

    if recursive_step == max_vector_size:
        yield row_list
    else:
        minimum_row_value = min_value

        if len(previous_row) > 0:
            minimum_row_value = max(min_value, previous_row[recursive_step])

        for i in range(minimum_row_value, max_value + 1):
            if min(recursive_step,row_position) <= i <= max(recursive_step,row_position):
                new_row = row_list.copy()
                new_row.append(i)
                yield from generate_row_compensation_space(previous_row=previous_row,
                                                           admissible_range=(i,max_value),
                                                           max_vector_size=max_vector_size,
                                                           row_position=row_position,
                                                           recursive_step=recursive_step+1,
                                                           row_list=new_row)
            else:
                continue
                
def generate_recursive_compensation_space(n: int, e: int, row: int, previous_rows: List):
    if row == n:
        yield previous_rows
    else:
        previous_boundary = previous_rows[len(previous_rows)-1]
        for next_row in generate_row_compensation_space(previous_row=previous_boundary, 
                                                        admissible_range=(0,row+1), 
                                                        max_vector_size=e,
                                                        row_position=row+1):
            if (row+1) == n:
                if next_row[0] == 0 or next_row[0] == n: #Every discrete uninorms satisfies that U(n,0)=0 or U(n,0)=n.
                    next_iteration_rows = previous_rows.copy()
                    next_iteration_rows.append(next_row)
                    yield from generate_recursive_compensation_space(n=n, e=e, row=row+1, previous_rows=next_iteration_rows)
                else:
                    continue
            else:
                next_iteration_rows = previous_rows.copy()
                next_iteration_rows.append(next_row)
                yield from generate_recursive_compensation_space(n=n, e=e, row=row+1, previous_rows=next_iteration_rows)
                
def generate_compensation_spaces(n: int, e: int):
    boundary = [i for i in range(0, e)]
    row = e+1
    
    for seed in generate_row_compensation_space(previous_row=boundary, 
                                                admissible_range=(0,row), 
                                                max_vector_size=e,
                                                row_position=row):

        for compensation_space in generate_recursive_compensation_space(n=n, e=e, row=row, previous_rows=[seed]):
            yield numpy.flipud(numpy.array(compensation_space))

SyntaxError: EOL while scanning string literal (2796552753.py, line 73)

In [3]:
discrete_dataset_path = r"C:\Users\Usuario\OneDrive - Universitat de les Illes Balears\UIB\Tesi\Experiments\DiscreteDataset"

available_tnorms = {}
available_tconorms = {}

for n in range(1, 8+1):
    available_tnorms[n]   = numpy.load(discrete_dataset_path+rf"\N={n}\tnorms.npy", allow_pickle=True)
    available_tconorms[n] = numpy.load(discrete_dataset_path+rf"\N={n}\tconorms.npy", allow_pickle=True)

In [3]:
def check_associativitiy(operator: numpy.ndarray, n: int) -> bool:
    for x in range(0, n+1):
        for y in range(0, n+1):
            for z in range(0, n+1):
                v1 = operator[y,z]
                v2 = operator[x,y]
                
                if operator[x, v1] != operator[v2, z]:
                    return False
    return True

def generate_uninorms(n: int, available_tnorms: Dict[int, List[numpy.array]], available_tconorms: Dict[int, List[numpy.array]], verbose: bool = False) -> Tuple[List[numpy.array], float]:
    verboseprint = print if verbose else lambda *a, **k: None
    
    available_uninorms = []
    start_time = time.time()

    for e in range(0, n+1):
        num_e = 0
        if e == 0:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            verboseprint(len(list(available_tnorms[n])))
            available_uninorms = available_uninorms+list(available_tnorms[n])
        elif e == n:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            verboseprint(len(list(available_tconorms[n])))
            available_uninorms = available_uninorms+list(available_tconorms[n])
        else:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            compensation_spaces = [space for space in generate_compensation_spaces(n=n, e=e)]
            
            for compensation_space in generate_compensation_spaces(n=n, e=e):
                for underlying_tnorm in available_tnorms[e]:
                    for underlying_tconorm in available_tconorms[n-e]:
                        compensation_space_mod = numpy.flipud(compensation_space)

                        uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                        uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                        uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                        uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                        uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                        if check_associativitiy(operator=uninorm_candidate, n=n):
                            available_uninorms.append(uninorm_candidate)
                            num_e += 1
            verboseprint(num_e)
    elapsed_time = time.time()-start_time
    
    return available_uninorms, elapsed_time

def count_uninorms(n: int, available_tnorms: Dict[int, List[numpy.array]], available_tconorms: Dict[int, List[numpy.array]], verbose: bool = False) -> Tuple[List[numpy.array], float]:
    verboseprint = print if verbose else lambda *a, **k: None
    
    available_uninorms = 0
    start_time = time.time()
    
    if n%2 == 0:
        for e in range(0, int(n/2+1)):
            if e == 0:
                verboseprint(f"---------- WORKING WITH e={e} AND e={n} ----------")
                verboseprint(2*len(list(available_tnorms[n])))
                available_uninorms = available_uninorms+2*len(list(available_tnorms[n]))
            else:
                available_uninorms_e = 0
                factor = 2
                if e == (int(n/2+1)-1):
                    factor = 1
                
                verboseprint(f"---------- WORKING WITH e={e} AND e={n-e} ----------")
                compensation_spaces = [space for space in generate_compensation_spaces(n=n, e=e)]
                
                for compensation_space in compensation_spaces:
                    for underlying_tnorm in available_tnorms[e]:
                        for underlying_tconorm in available_tconorms[n-e]:
                            compensation_space_mod = numpy.flipud(compensation_space)

                            uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                            uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                            uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                            uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                            uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                            if check_associativitiy(operator=uninorm_candidate, n=n):
                                available_uninorms_e += 1
                verboseprint(factor*available_uninorms_e)
                available_uninorms += factor*available_uninorms_e
    else:
        for e in range(0, int((n+1)/2)):
            if e == 0:
                verboseprint(f"---------- WORKING WITH e={e} AND e={n} ----------")
                verboseprint(2*len(list(available_tnorms[n])))
                available_uninorms = available_uninorms+2*len(list(available_tnorms[n]))
            else:
                available_uninorms_e = 0
                factor = 2
                
                verboseprint(f"---------- WORKING WITH e={e} AND e={n-e} ----------")
                compensation_spaces = [space for space in generate_compensation_spaces(n=n, e=e)]
                
                for compensation_space in compensation_spaces:
                    for underlying_tnorm in available_tnorms[e]:
                        for underlying_tconorm in available_tconorms[n-e]:
                            compensation_space_mod = numpy.flipud(compensation_space)

                            uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                            uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                            uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                            uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                            uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                            if check_associativitiy(operator=uninorm_candidate, n=n):
                                available_uninorms_e += 1
                verboseprint(factor*available_uninorms_e)
                available_uninorms += factor*available_uninorms_e
    
    elapsed_time = time.time()-start_time
    
    return available_uninorms, elapsed_time

In [4]:
def detect_neutral_element(uninorm: numpy.ndarray, n: int) -> int:
    for e in range(0, n+1):
        is_neutral_element = True
        x = 0
        while (x <= n and is_neutral_element):
            if uninorm[x,e] != x:
                is_neutral_element = False
            x = x+1
        if is_neutral_element:
            return e            

In [5]:
uninorms_n1, time_n1 = generate_uninorms(n=1, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={1} THERE ARE {len(uninorms_n1)} UNINORMS. ELAPSED TIME: {round(time_n1, 4)} seconds.")

NameError: name 'available_tnorms' is not defined

In [14]:
uninorms_n2, time_n2 = generate_uninorms(n=2, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={2} THERE ARE {len(uninorms_n2)} UNINORMS. ELAPSED TIME: {round(time_n2, 4)} seconds.")

---------- WORKING WITH e=0 ----------
2
---------- WORKING WITH e=1 ----------
2
---------- WORKING WITH e=2 ----------
2
WITH n=2 THERE ARE 6 UNINORMS. ELAPSED TIME: 0.001 seconds.


In [13]:
uninorms_n3, time_n3 = generate_uninorms(n=3, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={3} THERE ARE {len(uninorms_n3)} UNINORMS. ELAPSED TIME: {round(time_n3, 4)} seconds.")

---------- WORKING WITH e=0 ----------
6
---------- WORKING WITH e=1 ----------
5
---------- WORKING WITH e=2 ----------
5
---------- WORKING WITH e=3 ----------
6
WITH n=3 THERE ARE 22 UNINORMS. ELAPSED TIME: 0.0021 seconds.


In [15]:
uninorms_n4, time_n4 = generate_uninorms(n=4, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={4} THERE ARE {len(uninorms_n4)} UNINORMS. ELAPSED TIME: {round(time_n4, 4)} seconds.")

---------- WORKING WITH e=0 ----------
22
---------- WORKING WITH e=1 ----------
17
---------- WORKING WITH e=2 ----------
14
---------- WORKING WITH e=3 ----------
17
---------- WORKING WITH e=4 ----------
22
WITH n=4 THERE ARE 92 UNINORMS. ELAPSED TIME: 0.016 seconds.


In [16]:
uninorms_n5, time_n5 = generate_uninorms(n=5, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={5} THERE ARE {len(uninorms_n5)} UNINORMS. ELAPSED TIME: {round(time_n5, 4)} seconds.")

---------- WORKING WITH e=0 ----------
94
---------- WORKING WITH e=1 ----------
68
---------- WORKING WITH e=2 ----------
51
---------- WORKING WITH e=3 ----------
51
---------- WORKING WITH e=4 ----------
68
---------- WORKING WITH e=5 ----------
94
WITH n=5 THERE ARE 426 UNINORMS. ELAPSED TIME: 0.1826 seconds.


In [31]:
uninorms_n6, time_n6 = generate_uninorms(n=6, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={6} THERE ARE {len(uninorms_n6)} UNINORMS. ELAPSED TIME: {round(time_n6, 4)} seconds.")

---------- WORKING WITH e=0 ----------
451
---------- WORKING WITH e=1 ----------
308
---------- WORKING WITH e=2 ----------
217
---------- WORKING WITH e=3 ----------
194
---------- WORKING WITH e=4 ----------
217
---------- WORKING WITH e=5 ----------
308
---------- WORKING WITH e=6 ----------
451
WITH n=6 THERE ARE 2146 UNINORMS. ELAPSED TIME: 5.4807 seconds.


In [43]:
uninorms_n7, time_n7 = generate_uninorms(n=7, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(f"WITH n={7} THERE ARE {len(uninorms_n7)} UNINORMS. ELAPSED TIME: {round(time_n7, 4)} seconds.")

---------- WORKING WITH e=0 ----------
2386
---------- WORKING WITH e=1 ----------
1538
---------- WORKING WITH e=2 ----------
1034
---------- WORKING WITH e=3 ----------
859
---------- WORKING WITH e=4 ----------
859
---------- WORKING WITH e=5 ----------
1034
---------- WORKING WITH e=6 ----------
1538
---------- WORKING WITH e=7 ----------
2386
WITH n=7 THERE ARE 11634 UNINORMS. ELAPSED TIME: 347.896 seconds.


In [46]:
total_uninorms = {1: uninorms_n1, 2: uninorms_n2, 3: uninorms_n3, 4: uninorms_n4, 5: uninorms_n5, 6: uninorms_n6, 7: uninorms_n7}

In [53]:
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_operator import DiscreteAggregationBinaryOperator

def operator_is_idempotentfree(operator: numpy.ndarray, n: int) -> bool:
    if n == 1:
        return True
    else:
        for x in range(1, (n-1)+1):
            if operator[x,x] == x:
                return False
        return True

def operator_is_idempotent(operator: numpy.ndarray, n: int) -> bool:
    if n == 1:
        return True
    else:
        for x in range(1, (n-1)+1):
            if operator[x,x] != x:
                return False
        return True

def uninorm_smooth_underlying_operators(uninorm: numpy.ndarray, n: int):
    e = detect_neutral_element(uninorm=uninorm, n=n)
    
    if e == 0 or e == n:
        tnorm_tconorm = DiscreteAggregationBinaryOperator(n=n, operator_matrix=uninorm)
        return tnorm_tconorm.is_smooth()
   
    else:
        underlying_tnorm = uninorm[0:(e+1), 0:(e+1)]
        underlying_tconorm = uninorm[e:(n+1), e:(n+1)]-e
        
        tnorm = DiscreteAggregationBinaryOperator(n=e, operator_matrix=underlying_tnorm)
        tconorm = DiscreteAggregationBinaryOperator(n=n-e, operator_matrix=underlying_tconorm)

        return (tnorm.is_smooth() and tconorm.is_smooth())
    
def classify_smooth_uninorms(uninorm: numpy.ndarray, n: int):
    e = detect_neutral_element(uninorm=uninorm, n=n)
    
    if e == 0 or e == n:
        return operator_is_idempotentfree(uninorm, n) or operator_is_idempotent(uninorm, n)
    else:
        underlying_tnorm = uninorm[0:(e+1), 0:(e+1)]
        underlying_tconorm = uninorm[e:(n+1), e:(n+1)]-e
        
        tnorm_class = operator_is_idempotentfree(underlying_tnorm, e) or operator_is_idempotent(underlying_tnorm, e)
        tconorm_class = operator_is_idempotentfree(underlying_tconorm, n-e) or operator_is_idempotent(underlying_tconorm, n-e)
        
        return tnorm_class and tconorm_class

In [54]:
for n in range(1, 7+1):
    uninorms = total_uninorms[n]
    smooth_underlying_operators = [uni for uni in uninorms if uninorm_smooth_underlying_operators(uni, n)]
    arch_ide_uninorms = [smt_uni for smt_uni in smooth_underlying_operators if classify_smooth_uninorms(smt_uni, n)]
    print("----------------------")
    print(f"WORKING WITH n={n}")
    print(f"Total number of uninorms: {len(uninorms)}")
    print(f"Total number of uninorms having smooth underlying operators: {len(smooth_underlying_operators)}")
    print(f"Total number of uninorms having smooth and Archimedean or smooth and idempotent underlying operators: {len(arch_ide_uninorms)}")

----------------------
WORKING WITH n=1
Total number of uninorms: 2
Total number of uninorms having smooth underlying operators: 2
Total number of uninorms having smooth and Archimedean or smooth and idempotent underlying operators: 2
----------------------
WORKING WITH n=2
Total number of uninorms: 6
Total number of uninorms having smooth underlying operators: 6
Total number of uninorms having smooth and Archimedean or smooth and idempotent underlying operators: 6
----------------------
WORKING WITH n=3
Total number of uninorms: 22
Total number of uninorms having smooth underlying operators: 18
Total number of uninorms having smooth and Archimedean or smooth and idempotent underlying operators: 14
----------------------
WORKING WITH n=4
Total number of uninorms: 92
Total number of uninorms having smooth underlying operators: 54
Total number of uninorms having smooth and Archimedean or smooth and idempotent underlying operators: 30
----------------------
WORKING WITH n=5
Total number o

In [65]:
n = 4
uninorms = total_uninorms[n]
smooth_underlying_operators = [uni for uni in uninorms if uninorm_smooth_underlying_operators(uni, n)]
uninorms_e1 = [uni for uni in smooth_underlying_operators if detect_neutral_element(uninorm=uni, n=n) == 1]
special_uninorms_e1  = [uni for uni in uninorms_e1 if classify_smooth_uninorms(uninorm=uni, n=n)]
print(len(special_uninorms_e1))

6


In [80]:

def second_generator(x14:int,x15:int,x16:int,x17:int,x18:int,n:int):
    for x19 in range(max(x14,x18), n+1):
        for x20 in range(max(x15,x19), n+1):
            for x21 in range(x16, n+1):
                for x22 in range(max(x17,x21), n+1):
                    for x23 in range(max(x18,x22), n+1):
                        for x24 in range(max(x19,x23), n+1):
                            yield x19, x20, x21, x22, x23, x24

n = 5
conjunctions = []

count = 0
for x1 in range(0, n+1):
    for x2 in range(x1, n+1):
        for x3 in range(x2, n+1):
            for x4 in range(x3, n+1):
                for x5 in range(x4, n+1):
                    for x6 in range(x1, n+1):
                        for x7 in range(max(x2,x6), n+1):
                            for x8 in range(max(x3,x7), n+1):
                                for x9 in range(max(x4,x8), n+1):
                                    for x10 in range(max(x5,x9), n+1):
                                        for x11 in range(x6, n+1):
                                            for x12 in range(max(x7,x11), n+1):
                                                for x13 in range(max(x8,x12), n+1):
                                                    for x14 in range(max(x9,x13), n+1):
                                                        for x15 in range(max(x10,x14), n+1):
                                                            for x16 in range(x11, n+1):
                                                                for x17 in range(max(x12,x16), n+1):
                                                                    for x18 in range(max(x13,x17), n+1):
                                                                        for x19, x20, x21, x22, x23, x24 in second_generator(x14,x15,x16,x17,x18,n):
                                                                            count += 1
                                                                            
                                                                            conjunction = numpy.zeros((n+1, n+1),dtype=numpy.int8)
                                                                            conjunction[1,1] = x1
                                                                            conjunction[1,2] = x2
                                                                            conjunction[1,3] = x3
                                                                            conjunction[1,4] = x4
                                                                            conjunction[1,5] = x5
                                                                            
                                                                            conjunction[2,1] = x6
                                                                            conjunction[2,2] = x7
                                                                            conjunction[2,3] = x8
                                                                            conjunction[2,4] = x9
                                                                            conjunction[2,5] = x10
                                                                            
                                                                            conjunction[3,1] = x11
                                                                            conjunction[3,2] = x12
                                                                            conjunction[3,3] = x13
                                                                            conjunction[3,4] = x14
                                                                            conjunction[3,5] = x15
                                                                            
                                                                            conjunction[4,1] = x16
                                                                            conjunction[4,2] = x17
                                                                            conjunction[4,3] = x18
                                                                            conjunction[4,4] = x19
                                                                            conjunction[4,5] = x20
                                                                            
                                                                            conjunction[5,1] = x21
                                                                            conjunction[5,2] = x22
                                                                            conjunction[5,3] = x23
                                                                            conjunction[5,4] = x24
                                                                            
                                                                            conjunctions.append(conjunction)
                                                                            
print(count)                                                                                     

TypeError: __init__() missing 2 required positional arguments: 'shape' and 'dtype'

In [71]:
numpy.save(r"C:\Users\Usuario\Desktop\conjunctions.npy", conjunctions, allow_pickle=True)

MemoryError: Unable to allocate 33.6 GiB for an array with shape (250409016, 6, 6) and data type int32

In [75]:
print(len(conjunctions))

250409016
